# 🎙️ Podcast Parser & Transkriptor

Ein umfassendes Tool zum Herunterladen von Podcast-Folgen aus RSS-Feeds und deren Transkription mit OpenAI's Whisper-Modell.

---

## 📋 Features

### 📥 Podcast Downloader
- RSS-Feed-Parsing und Analyse
- Selektiver Download von Folgen
- Dateigrößen-Anzeige
- Intelligente Dateinamen-Generierung

### 📝 Audio Transkriptor
- OpenAI Whisper Integration
- Mehrsprachige Transkription (Deutsch, Englisch, Auto)
- Verschiedene Modell-Größen (tiny bis large)
- Multi-Format Support (MP3, WAV, M4A, FLAC, OGG)

---

## 🚀 Installation & Setup

In [ ]:
# Install required packages
!pip install feedparser requests openai-whisper

# Import libraries
import os
import requests
import feedparser
import whisper
import tempfile
import shutil
from urllib.parse import urlparse
from google.colab import files
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets
from datetime import datetime

print("✅ Alle Abhängigkeiten installiert und importiert!")

## 🛠️ Hilfsfunktionen

In [ ]:
def title_to_filename(title, ext):
    """Konvertiert einen Titel in einen gültigen Dateinamen"""
    invalid_chars = ['?', '*', ':', '<', '>', '|', '"', '\\', '/', '\n', '\r']
    filename = title
    for char in invalid_chars:
        filename = filename.replace(char, "_")
    filename = filename.strip().replace(' ', '_')
    filename = filename[:120]  # Länge beschränken
    return f"{filename}{ext}"

def format_file_size(size_bytes):
    """Formatiert Bytes in lesbare Größe"""
    if size_bytes == 0:
        return "0 B"
    size_names = ["B", "KB", "MB", "GB"]
    import math
    i = int(math.floor(math.log(size_bytes, 1024)))
    p = math.pow(1024, i)
    s = round(size_bytes / p, 2)
    return f"{s} {size_names[i]}"

def create_download_link(filename, text):
    """Erstellt einen Download-Link für Colab"""
    html = f'<a href="files/{filename}" download="{filename}">{text}</a>'
    return HTML(html)

print("✅ Hilfsfunktionen definiert!")

## 📥 Podcast Downloader

In [ ]:
# Widgets für Podcast Downloader
feed_url_widget = widgets.Text(
    value="https://www.energiezone.org/feed/mp3",
    description="RSS-Feed URL:",
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='600px')
)

download_dir_widget = widgets.Text(
    value="./podcasts",
    description="Zielordner:",
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='600px')
)

load_feed_button = widgets.Button(
    description="📡 Feed laden & Folgen anzeigen",
    button_style='primary',
    layout=widgets.Layout(width='300px')
)

download_button = widgets.Button(
    description="⬇️ Ausgewählte Folgen herunterladen",
    button_style='success',
    layout=widgets.Layout(width='300px')
)

# Container für Checkboxen
episode_checkboxes = widgets.VBox([])
episode_data = []

display(HTML("<h3>🎙️ Podcast Downloader</h3>"))
display(feed_url_widget)
display(download_dir_widget)
display(load_feed_button)
display(episode_checkboxes)
display(download_button)

In [ ]:
def load_feed(b):
    print("Feed laden wurde geklickt!")
    # Hier kommt dein Code zum Laden des Feeds

def download_episodes(b):
    print("Download wurde geklickt!")
    # Hier kommt dein Code zum Herunterladen der Episoden

# Event-Handler verbinden
load_feed_button.on_click(load_feed)
download_button.on_click(download_episodes)


## 📝 Audio Transkriptor

In [ ]:
# Widgets für Audio Transkriptor
model_size_widget = widgets.Dropdown(
    options=["tiny", "base", "small", "medium", "large"],
    value="base",
    description="Whisper Modell:",
    style={'description_width': 'initial'}
)

load_model_button = widgets.Button(
    description="🤖 Modell laden",
    button_style='primary'
)

language_widget = widgets.Dropdown(
    options=["auto", "de", "en"],
    value="auto",
    description="Sprache:",
    style={'description_width': 'initial'}
)

transcribe_button = widgets.Button(
    description="🎯 Transkribieren",
    button_style='success'
)

model_status = widgets.HTML(value="<span style='color: red;'>❌ Modell nicht geladen</span>")

display(HTML("<h3>📝 Audio Transkriptor</h3>"))
display(model_size_widget)
display(load_model_button)
display(model_status)
display(language_widget)
display(transcribe_button)

In [ ]:
def load_whisper_model(b):
    print("Modell laden wurde geklickt!")
    # Hier kommt dein Code zum Laden des Whisper-Modells

def transcribe_audio(b):
    print("Transkribieren wurde geklickt!")
    # Hier kommt dein Code zur Transkription

# Event-Handler verbinden
load_model_button.on_click(load_whisper_model)
transcribe_button.on_click(transcribe_audio)


## 🔄 Batch-Transkription für heruntergeladene Podcasts

In [ ]:
# Widgets für Batch-Transkription
batch_dir_widget = widgets.Text(
    value="./podcasts",
    description="Podcast-Ordner:",
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='600px')
)

batch_language_widget = widgets.Dropdown(
    options=["auto", "de", "en"],
    value="auto",
    description="Sprache:",
    style={'description_width': 'initial'}
)

batch_transcribe_button = widgets.Button(
    description="🔄 Alle Podcasts transkribieren",
    button_style='warning'
)

display(HTML("<h3>🔄 Batch-Transkription</h3>"))
display(HTML("<p>Transkribiert alle Audio-Dateien in einem Ordner automatisch.</p>"))
display(batch_dir_widget)
display(batch_language_widget)
display(batch_transcribe_button)

In [ ]:
def batch_transcribe(b):
    """Transkribiert alle Audio-Dateien in einem Ordner"""
    global whisper_model
    
    if whisper_model is None:
        print("❌ Bitte laden Sie zuerst ein Whisper-Modell!")
        return
    
    batch_dir = batch_dir_widget.value
    language = batch_language_widget.value
    
    if not os.path.exists(batch_dir):
        print(f"❌ Ordner nicht gefunden: {batch_dir}")
        return
    
    # Finde alle Audio-Dateien
    audio_extensions = ('.mp3', '.wav', '.m4a', '.flac', '.ogg')
    audio_files = [f for f in os.listdir(batch_dir) 
                   if f.lower().endswith(audio_extensions)]
    
    if not audio_files:
        print(f"❌ Keine Audio-Dateien in {batch_dir} gefunden!")
        return
    
    print(f"🔄 Starte Batch-Transkription für {len(audio_files)} Dateien...")
    
    # Erstelle Ausgabe-Ordner
    output_dir = f"{batch_dir}_transkriptionen"
    os.makedirs(output_dir, exist_ok=True)
    
    for idx, audio_file in enumerate(audio_files):
        audio_path = os.path.join(batch_dir, audio_file)
        original_filename = os.path.splitext(audio_file)[0]
        output_filename = f"{original_filename}.txt"
        output_path = os.path.join(output_dir, output_filename)
        
        # Prüfe ob Transkription bereits existiert
        if os.path.exists(output_path):
            print(f"⏭️ {audio_file} bereits transkribiert, überspringe.")
            continue
        
        print(f"🎵 [{idx+1}/{len(audio_files)}] Transkribiere: {audio_file}")
        
        try:
            # Führe Transkription durch
            result = whisper_model.transcribe(
                audio_path,
                language=language if language != "auto" else None,
                task="transcribe"
            )
            
            # Speichere Transkription
            with open(output_path, 'w', encoding='utf-8') as f:
                f.write(result["text"])
            
            print(f"✅ Gespeichert: {output_filename}")
            
        except Exception as e:
            print(f"❌ Fehler bei {audio_file}: {e}")
    
    print(f"\n🎉 Batch-Transkription abgeschlossen!")
    print(f"📁 Transkriptionen gespeichert in: {output_dir}")
    
    # Erstelle ZIP-Archiv für einfachen Download
    import zipfile
    zip_filename = f"{batch_dir}_transkriptionen.zip"
    
    with zipfile.ZipFile(zip_filename, 'w') as zipf:
        for file in os.listdir(output_dir):
            file_path = os.path.join(output_dir, file)
            zipf.write(file_path, file)
    
    print(f"📦 ZIP-Archiv erstellt: {zip_filename}")
    display(create_download_link(zip_filename, "⬇️ Alle Transkriptionen herunterladen (ZIP)"))

# Event-Handler verbinden
batch_transcribe_button.on_click(batch_transcribe)

print("✅ Batch-Transkription Widgets erstellt!")

## 📖 Verwendungsbeispiele

In [ ]:
# Beispiel: Automatischer Workflow
def example_workflow():
    """Demonstriert einen typischen Workflow"""
    
    print("🚀 Beispiel-Workflow: Podcast herunterladen und transkribieren")
    print("="*60)
    
    # 1. Feed laden
    print("1️⃣ RSS-Feed laden...")
    print("   - Geben Sie die RSS-URL ein")
    print("   - Klicken Sie auf 'Feed laden & Folgen anzeigen'")
    print("   - Wählen Sie gewünschte Folgen aus")
    print()
    
    # 2. Folgen herunterladen
    print("2️⃣ Folgen herunterladen...")
    print("   - Klicken Sie auf 'Ausgewählte Folgen herunterladen'")
    print("   - Warten Sie bis alle Downloads abgeschlossen sind")
    print()
    
    # 3. Whisper-Modell laden
    print("3️⃣ Whisper-Modell laden...")
    print("   - Wählen Sie Modell-Größe (tiny = schnell, large = genau)")
    print("   - Klicken Sie auf 'Modell laden'")
    print()
    
    # 4. Transkribieren
    print("4️⃣ Audio transkribieren...")
    print("   - Option A: Einzelne Datei hochladen")
    print("   - Option B: Batch-Transkription für alle heruntergeladenen Folgen")
    print()
    
    # 5. Ergebnisse
    print("5️⃣ Ergebnisse verwenden...")
    print("   - Transkriptionen als Text herunterladen")
    print("   - Für weitere Verarbeitung verwenden")
    print("="*60)

# Zeige Beispiel
example_workflow()

## 💡 Tipps & Fehlerbehebung

### 🎯 Für beste Transkriptions-Ergebnisse:
- **Audioqualität**: Verwenden Sie hochwertige Audio-Dateien
- **Hintergrundgeräusche**: Reduzieren Sie Störgeräusche
- **Modellauswahl**: 
  - **Tiny/Base**: Für schnelle Transkription
  - **Medium/Large**: Für hohe Genauigkeit
- **Sprache**: Wählen Sie die korrekte Sprache
- **RAM**: Stellen Sie sicher, dass genügend RAM verfügbar ist

### 🔧 Häufige Probleme:

**Modell kann nicht geladen werden:**
- Überprüfen Sie Ihre Internetverbindung (erster Download)
- Stellen Sie sicher, dass genügend RAM verfügbar ist
- Versuchen Sie ein kleineres Modell

**Audio-Datei kann nicht verarbeitet werden:**
- Überprüfen Sie, ob die Datei beschädigt ist
- Stellen Sie sicher, dass das Format unterstützt wird
- Versuchen Sie eine andere Audio-Datei

**Langsame Performance:**
- Verwenden Sie ein kleineres Whisper-Modell
- Schließen Sie andere Programme
- Stellen Sie sicher, dass genügend RAM verfügbar ist

### 📊 Whisper-Modell Vergleich:

| Modell | Größe   | RAM  | Geschwindigkeit | Genauigkeit | Empfehlung |
| ------ | ------- | ---- | --------------- | ----------- | ---------- |
| tiny   | 39 MB   | 1GB  | Sehr schnell    | Gut         | Schnelle Tests |
| base   | 74 MB   | 1GB  | Schnell         | Besser      | Standard |
| small  | 244 MB  | 2GB  | Mittel          | Noch besser | Gute Balance |
| medium | 769 MB  | 5GB  | Langsam         | Sehr gut    | Hohe Qualität |
| large  | 1550 MB | 10GB | Sehr langsam    | Beste       | Beste Qualität |

## 🎉 Fazit

Dieses Notebook bietet eine vollständige Lösung für:

✅ **Podcast-Download** aus RSS-Feeds  
✅ **Audio-Transkription** mit OpenAI Whisper  
✅ **Batch-Verarbeitung** für mehrere Dateien  
✅ **Benutzerfreundliche Oberfläche** mit Widgets  
✅ **Export-Funktionen** für Transkriptionen  

---

**Hinweis**: Stellen Sie sicher, dass Sie die Rechte haben, Podcast-Inhalte herunterzuladen und zu transkribieren. Respektieren Sie die Urheberrechte der Podcast-Ersteller.

**Inspiriert von**: [LocalTranscript](https://github.com/chradden/LocalTranscript)